In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data = pd.read_csv('nor.csv')  # 載入正規化後的資料

X = data.iloc[:,3 :-2].values
# 獲取標籤值
Y = data.iloc[:,[-1]].values
# 使用sklearn 的DecisionTreeClassifier判斷變數重要性
# 建立分類決策樹模型物件
dt_model = DecisionTreeClassifier(random_state=1)
# 將資料集的維度和目標變數輸入模型
dt_model.fit(X, Y)
# 獲取所有變數的重要性
feature_importance = dt_model.feature_importances_
print(feature_importance)

[0.11070416 0.05790777 0.00480714 0.03253976 0.         0.00874026
 0.0136683  0.         0.         0.01241845 0.03406202 0.02737196
 0.02889091 0.01970928 0.01304795 0.0238456  0.11144907 0.01695385
 0.02009138 0.00721071 0.00480714 0.02680798 0.         0.05246371
 0.14400497 0.02419841 0.03980662 0.02610249 0.01660066 0.0806927
 0.02493482 0.01616194]


In [3]:
# 做視覺化
import matplotlib.pyplot as plt

x = range(len(data.columns[3:-2])) 
# plt.figure(figsize=(36,4))
# plt.bar(left=x, height=feature_importance)
# plt.xticks(x, data.columns[3:-2])

In [4]:
#使用直接抽取特徵
# X = data[['AGE','FG' ,'TRB','GP','MPG', 'PIE']] 
# print(X.shape)

# 使用sklearn的PCA進行維度轉換
# 建立PCA模型物件 n_components控制輸出特徵個數
pca_model = PCA(n_components=6)
# 將資料集輸入模型
pca_model.fit(X)
# 對資料集進行轉換對映
X = pca_model.transform(X)
# 獲得轉換後的所有主成分
components = pca_model.components_
# 獲得各主成分的方差
components_var = pca_model.explained_variance_
# 獲取主成分的方差佔比
components_var_ratio = pca_model.explained_variance_ratio_
# 列印方差
print(np.round(components_var,5))
# 列印方差佔比
print(np.round(components_var_ratio,5))

print(X.shape)

[0.52565 0.14031 0.08317 0.0545  0.03813 0.03183]
[0.50784 0.13555 0.08035 0.05266 0.03684 0.03075]
(335, 6)


In [5]:
skf = StratifiedKFold(n_splits=4, random_state=2, shuffle=True)  # 將資料按照label比例分出train data和test data(分成4種)
accuracy = 0  # 計算平均accuracy
trainDataLen = 0  # 計算平均train data大小
testDataLen = 0  # 計算平均test data大小
confusion = np.zeros([3,3])  # 計算平均confusion matrix
r = 25  # range

player_id = []
predict_class = []
real_class = []
for i in range(r):
    print('第', i, '輪')
    model = Sequential()
    model.add(Dense(units=128, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=3, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    for train_index, test_index in skf.split(X, Y):
        
        trainDataLen = trainDataLen + len(train_index)
        testDataLen = testDataLen + len(test_index)
        y_TrainOneHot = np_utils.to_categorical(Y[train_index])
        y_TestOneHot = np_utils.to_categorical(Y[test_index])
        train_history = model.fit(x = X[train_index], y = y_TrainOneHot , epochs=100, verbose=1)
        scores = model.evaluate(X[test_index], y_TestOneHot)
        accuracy = accuracy + scores[1]
        y_predict = model.predict_classes(X[test_index])
        confusion = confusion + confusion_matrix(Y[test_index], y_predict)
        player_id.append(test_index)
        predict_class.append(y_predict)
        real_class.append(Y[test_index])

print('trainDataLen:', trainDataLen/(r*4), 'testDataLen:', testDataLen/(r*4))
print("Accuracy:{:2.1f}%".format(accuracy * 100.0 / (r*4)))
print('confusion:\n', confusion/(r*4))

第 0 輪
Epoch 1/100
250/250 [==============================] - 0s 858us/step - loss: 1.0173 - accuracy: 0.5200
Epoch 2/100
250/250 [==============================] - 0s 335us/step - loss: 0.8299 - accuracy: 0.6200
Epoch 3/100
250/250 [==============================] - 0s 311us/step - loss: 0.7257 - accuracy: 0.6560
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 0.6753 - accuracy: 0.70 - 0s 307us/step - loss: 0.6772 - accuracy: 0.6840
Epoch 5/100
250/250 [==============================] - 0s 327us/step - loss: 0.6206 - accuracy: 0.7120
Epoch 6/100
250/250 [==============================] - 0s 315us/step - loss: 0.6050 - accuracy: 0.7360
Epoch 7/100
250/250 [==============================] - 0s 361us/step - loss: 0.6212 - accuracy: 0.7080
Epoch 8/100
250/250 [==============================] - 0s 359us/step - loss: 0.5817 - accuracy: 0.7640
Epoch 9/100
250/250 [==============================] - 0s 351us/step - loss: 0.5748 - accuracy: 0.7520
Epoch 10/100
250/250 [===

250/250 [==============================] - 0s 299us/step - loss: 0.0860 - accuracy: 0.9800
Epoch 58/100
250/250 [==============================] - 0s 343us/step - loss: 0.0556 - accuracy: 0.9840
Epoch 59/100
250/250 [==============================] - 0s 303us/step - loss: 0.0419 - accuracy: 0.9840
Epoch 60/100
250/250 [==============================] - 0s 288us/step - loss: 0.0389 - accuracy: 0.9880
Epoch 61/100
250/250 [==============================] - 0s 291us/step - loss: 0.0339 - accuracy: 0.9920
Epoch 62/100
250/250 [==============================] - 0s 289us/step - loss: 0.0279 - accuracy: 0.9960
Epoch 63/100
250/250 [==============================] - 0s 357us/step - loss: 0.0313 - accuracy: 0.9920
Epoch 64/100
250/250 [==============================] - 0s 285us/step - loss: 0.0258 - accuracy: 1.0000
Epoch 65/100
250/250 [==============================] - 0s 291us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 66/100
250/250 [==============================] - 0s 307us/step - loss:

252/252 [==============================] - 0s 297us/step - loss: 0.0377 - accuracy: 0.9881
Epoch 36/100
252/252 [==============================] - 0s 310us/step - loss: 0.0448 - accuracy: 0.9841
Epoch 37/100
252/252 [==============================] - 0s 367us/step - loss: 0.0324 - accuracy: 0.9921
Epoch 38/100
252/252 [==============================] - 0s 294us/step - loss: 0.0343 - accuracy: 0.9960
Epoch 39/100
252/252 [==============================] - 0s 304us/step - loss: 0.0362 - accuracy: 0.9881
Epoch 40/100
252/252 [==============================] - 0s 301us/step - loss: 0.0199 - accuracy: 0.9960
Epoch 41/100
252/252 [==============================] - 0s 293us/step - loss: 0.0186 - accuracy: 0.9960
Epoch 42/100
252/252 [==============================] - 0s 300us/step - loss: 0.0307 - accuracy: 0.9881
Epoch 43/100
252/252 [==============================] - 0s 305us/step - loss: 0.0240 - accuracy: 0.9881
Epoch 44/100
252/252 [==============================] - 0s 297us/step - loss:

253/253 [==============================] - 0s 430us/step - loss: 0.0113 - accuracy: 0.9960
Epoch 92/100
253/253 [==============================] - 0s 520us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 93/100
253/253 [==============================] - 0s 673us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 94/100
253/253 [==============================] - 0s 463us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 95/100
253/253 [==============================] - 0s 412us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 96/100
253/253 [==============================] - 0s 398us/step - loss: 0.0099 - accuracy: 0.9960
Epoch 97/100
253/253 [==============================] - 0s 371us/step - loss: 0.0045 - accuracy: 1.0000
Epoch 98/100
253/253 [==============================] - 0s 313us/step - loss: 0.0045 - accuracy: 1.0000
Epoch 99/100
253/253 [==============================] - 0s 382us/step - loss: 0.0049 - accuracy: 1.0000
Epoch 100/100
82/82 [==============================] - 0s 122us/step
第 1 輪
Ep

250/250 [==============================] - 0s 327us/step - loss: 0.2137 - accuracy: 0.9120
Epoch 70/100
250/250 [==============================] - 0s 331us/step - loss: 0.1673 - accuracy: 0.9320
Epoch 71/100
250/250 [==============================] - 0s 319us/step - loss: 0.2024 - accuracy: 0.9200
Epoch 72/100
250/250 [==============================] - 0s 319us/step - loss: 0.1596 - accuracy: 0.9440
Epoch 73/100
250/250 [==============================] - 0s 318us/step - loss: 0.1875 - accuracy: 0.9120
Epoch 74/100
250/250 [==============================] - 0s 315us/step - loss: 0.1926 - accuracy: 0.9160
Epoch 75/100
250/250 [==============================] - 0s 306us/step - loss: 0.1572 - accuracy: 0.9320
Epoch 76/100
250/250 [==============================] - 0s 304us/step - loss: 0.1604 - accuracy: 0.9320
Epoch 77/100
250/250 [==============================] - 0s 318us/step - loss: 0.1417 - accuracy: 0.9560
Epoch 78/100
250/250 [==============================] - 0s 324us/step - loss:

...

250/250 [==============================] - 0s 383us/step - loss: 0.0644 - accuracy: 0.9840
Epoch 51/100
250/250 [==============================] - 0s 395us/step - loss: 0.1130 - accuracy: 0.9640
Epoch 52/100
250/250 [==============================] - 0s 352us/step - loss: 0.0628 - accuracy: 0.9880
Epoch 53/100
250/250 [==============================] - 0s 290us/step - loss: 0.0568 - accuracy: 0.9800
Epoch 54/100
250/250 [==============================] - 0s 312us/step - loss: 0.0471 - accuracy: 0.9840
Epoch 55/100
250/250 [==============================] - 0s 360us/step - loss: 0.0420 - accuracy: 0.9880
Epoch 56/100
250/250 [==============================] - 0s 407us/step - loss: 0.0489 - accuracy: 0.9880
Epoch 57/100
250/250 [==============================] - 0s 359us/step - loss: 0.0557 - accuracy: 0.9840
Epoch 58/100
250/250 [==============================] - 0s 349us/step - loss: 0.0566 - accuracy: 0.9840
Epoch 59/100
250/250 [==============================] - 0s 355us/step - loss:

253/253 [==============================] - 0s 371us/step - loss: 0.0745 - accuracy: 0.9723
Epoch 7/100
253/253 [==============================] - 0s 356us/step - loss: 0.0547 - accuracy: 0.9881
Epoch 8/100
253/253 [==============================] - 0s 355us/step - loss: 0.0319 - accuracy: 0.9842
Epoch 9/100
253/253 [==============================] - 0s 359us/step - loss: 0.0400 - accuracy: 0.9842
Epoch 10/100
253/253 [==============================] - 0s 351us/step - loss: 0.0223 - accuracy: 0.9960
Epoch 11/100
253/253 [==============================] - 0s 357us/step - loss: 0.0478 - accuracy: 0.9763
Epoch 12/100
253/253 [==============================] - 0s 375us/step - loss: 0.0184 - accuracy: 1.0000
Epoch 13/100
253/253 [==============================] - 0s 371us/step - loss: 0.0791 - accuracy: 0.9802
Epoch 14/100
253/253 [==============================] - 0s 358us/step - loss: 0.0693 - accuracy: 0.9802
Epoch 15/100
253/253 [==============================] - 0s 355us/step - loss: 0.

250/250 [==============================] - 0s 423us/step - loss: 0.2355 - accuracy: 0.9080
Epoch 63/100
250/250 [==============================] - 0s 403us/step - loss: 0.2452 - accuracy: 0.9040
Epoch 64/100
250/250 [==============================] - 0s 388us/step - loss: 0.2022 - accuracy: 0.9280
Epoch 65/100
250/250 [==============================] - 0s 411us/step - loss: 0.2048 - accuracy: 0.9200
Epoch 66/100
250/250 [==============================] - 0s 389us/step - loss: 0.2284 - accuracy: 0.9240
Epoch 67/100
250/250 [==============================] - 0s 443us/step - loss: 0.1926 - accuracy: 0.9120
Epoch 68/100
250/250 [==============================] - 0s 377us/step - loss: 0.2196 - accuracy: 0.9040
Epoch 69/100
250/250 [==============================] - 0s 387us/step - loss: 0.1898 - accuracy: 0.9200
Epoch 70/100
250/250 [==============================] - 0s 407us/step - loss: 0.2036 - accuracy: 0.9280
Epoch 71/100
250/250 [==============================] - 0s 415us/step - loss:

252/252 [==============================] - 0s 400us/step - loss: 0.0984 - accuracy: 0.9603
Epoch 19/100
252/252 [==============================] - 0s 354us/step - loss: 0.0755 - accuracy: 0.9643
Epoch 20/100
252/252 [==============================] - 0s 400us/step - loss: 0.0726 - accuracy: 0.9683
Epoch 21/100
252/252 [==============================] - 0s 384us/step - loss: 0.0403 - accuracy: 0.9881
Epoch 22/100
252/252 [==============================] - 0s 368us/step - loss: 0.0598 - accuracy: 0.9802
Epoch 23/100
252/252 [==============================] - 0s 351us/step - loss: 0.0380 - accuracy: 0.9881
Epoch 24/100
252/252 [==============================] - 0s 384us/step - loss: 0.0326 - accuracy: 0.9841
Epoch 25/100
252/252 [==============================] - 0s 412us/step - loss: 0.0382 - accuracy: 0.9921
Epoch 26/100
252/252 [==============================] - 0s 416us/step - loss: 0.0247 - accuracy: 1.0000
Epoch 27/100
252/252 [==============================] - 0s 392us/step - loss:

253/253 [==============================] - 0s 371us/step - loss: 0.0056 - accuracy: 0.9960
Epoch 75/100
253/253 [==============================] - 0s 355us/step - loss: 0.0121 - accuracy: 0.9921
Epoch 76/100
253/253 [==============================] - 0s 351us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 77/100
253/253 [==============================] - 0s 371us/step - loss: 0.0123 - accuracy: 0.9960
Epoch 78/100
253/253 [==============================] - 0s 390us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 79/100
253/253 [==============================] - 0s 359us/step - loss: 0.0035 - accuracy: 1.0000
Epoch 80/100
253/253 [==============================] - 0s 363us/step - loss: 0.0083 - accuracy: 0.9960
Epoch 81/100
253/253 [==============================] - 0s 374us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 82/100
253/253 [==============================] - 0s 398us/step - loss: 0.0057 - accuracy: 1.0000
Epoch 83/100
253/253 [==============================] - 0s 396us/step - loss:

In [7]:
playerFalseTime = {}#被預測錯誤幾次
playerFalse = {}#被錯誤預測成哪個class
playerTrue = {}#正確class
for i in range(len(player_id)):
    for x in range(len(player_id[i])):
        if predict_class[i][x] != real_class[i][x][0]:
            playerFalseTime[player_id[i][x]] = playerFalseTime.get(player_id[i][x],0)+1
            playerFalse[player_id[i][x]] = playerFalse.get(player_id[i][x],0)+predict_class[i][x]
            playerTrue[player_id[i][x]] = real_class[i][x][0]
        

In [8]:
for p in playerFalse:
    playerFalse[p] = playerFalse[p]/playerFalseTime[p]

In [9]:
player_name = data['PLAYER']
false_player_name = []
#從分類錯誤的index值對照原始data找出球員名字
for x in playerFalse:
    false_player_name.append(player_name[x])

In [10]:
estimate = []
predict = []
real = []
times = []
for p in playerFalse:
    if playerFalse[p]>playerTrue[p]:
        estimate.append(0) #低估
    else:
        estimate.append(1) #高估
for p in playerFalse:
    predict.append(playerFalse[p])
for p in playerTrue:
    real.append(playerTrue[p])
for p in playerFalseTime:
    times.append(playerFalseTime[p])

In [11]:
#建立dataframe for 機器分類錯誤的球員
dict = {'PLAYER': false_player_name, 'predict': predict, 'real': real,'estimate': estimate,'times':times}  
all_false_player = pd.DataFrame(dict) 
all_false_player

,PLAYER,predict,real,estimate,times
0,Giannis Antetokounmpo,2.000000,1,0,24
1,Joel Embiid,1.894737,1,0,19
2,Danilo Gallinari,1.000000,2,1,25
3,Dion Waiters,2.000000,1,0,25
4,Bojan Bogdanovic,2.000000,1,0,25
5,Trevor Ariza,2.000000,1,0,25
6,Greg Monroe,0.187500,2,1,16
7,Marcin Gortat,1.000000,2,1,20
8,Allen Crabbe,1.000000,2,1,25
9,Tony Parker,1.000000,2,1,25


In [12]:
#抓出低估/高估球員分別拿出來看
under = all_false_player['estimate'] == 0
over = all_false_player['estimate'] == 1

In [13]:
all_false_player[under]

,PLAYER,predict,real,estimate,times
0,Giannis Antetokounmpo,2.000000,1,0,24
1,Joel Embiid,1.894737,1,0,19
3,Dion Waiters,2.000000,1,0,25
4,Bojan Bogdanovic,2.000000,1,0,25
5,Trevor Ariza,2.000000,1,0,25
10,Trevor Booker,2.000000,1,0,25
11,Monta Ellis,2.000000,1,0,18
12,Thabo Sefolosha,2.000000,1,0,19
13,Justin Anderson,1.000000,0,0,25
14,Tim Frazier,1.000000,0,0,21


In [14]:
all_false_player[over]

,PLAYER,predict,real,estimate,times
2,Danilo Gallinari,1.000000,2,1,25
6,Greg Monroe,0.187500,2,1,16
7,Marcin Gortat,1.000000,2,1,20
8,Allen Crabbe,1.000000,2,1,25
9,Tony Parker,1.000000,2,1,25
15,John Henson,1.000000,2,1,25
17,Chandler Parsons,0.760000,2,1,25
18,Ian Mahinmi,1.000000,2,1,21
20,Dragan Bender,0.000000,1,1,25
23,Miles Plumlee,1.000000,2,1,25


In [15]:
data2 = pd.read_csv('nba_2017_nba_players_with_salary.csv')

In [16]:
# # 刪除重複的球員(數據也都一樣)
data2 = data2.drop_duplicates(['PLAYER'],keep='last')
data2.reset_index(drop=True, inplace=True)

# 刪除不需要之features，在這裡看些簡單數據資料就好('TEAM','POSITION','AGE','POINTS','FG%','TRB','GP','MPG', 'PIE','SALARY_MILLIONS')
data2 = data2.drop(['MP','Unnamed: 0','Rk','FGA','FG','3P', '3PA', '3P%','2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'AST','STL','BLK','TOV','PF','ORPM',
       'DRPM','RPM','WINS_RPM','PACE','W'], axis=1)

#合併我們的分析與原資料(未normalized的)
#低估(under_estimate)的table
under_estimate_player = pd.merge(data2, all_false_player[under], on=['PLAYER'])
#高估(over_estimate)的table
over_estimate_player = pd.merge(data2, all_false_player[over], on=['PLAYER'])

In [17]:
#低估(under_estimate)的table
under_estimate_player
#len(result0)

,PLAYER,POSITION,AGE,FG%,TRB,POINTS,TEAM,GP,MPG,PIE,SALARY_MILLIONS,predict,real,estimate,times
0,Isaiah Thomas,PG,27,0.463,2.7,28.9,BOS,76,33.8,16.1,6.59,2.000000,1,0,25
1,Karl-Anthony Towns,C,21,0.542,12.3,25.1,MIN,82,37.0,17.1,5.96,2.000000,1,0,1
2,Giannis Antetokounmpo,SF,22,0.521,8.8,22.9,MIL,80,35.6,17.6,3.00,2.000000,1,0,24
3,Joel Embiid,C,22,0.466,7.8,20.2,PHI,31,25.4,17.4,4.83,1.894737,1,0,19
4,Zach LaVine,SG,21,0.459,3.4,18.9,MIN,47,37.2,9.5,2.24,2.000000,1,0,4
5,Dennis Schroder,PG,23,0.451,3.1,17.9,ATL,79,31.5,11.7,2.71,2.000000,1,0,8
6,George Hill,PG,30,0.477,3.4,16.9,UTAH,49,31.5,12.8,8.00,2.000000,1,0,25
7,Nikola Jokic,C,21,0.578,9.8,16.7,DEN,73,27.9,16.7,1.36,1.250000,0,0,4
8,Avery Bradley,SG,26,0.463,6.1,16.3,BOS,55,33.4,9.9,8.27,2.000000,1,0,5
9,Victor Oladipo,SG,24,0.442,4.3,15.9,OKC,67,33.2,9.5,6.55,2.000000,1,0,17


In [18]:
#高估(over_estimate)的table
over_estimate_player
#len(result1)

,PLAYER,POSITION,AGE,FG%,TRB,POINTS,TEAM,GP,MPG,PIE,SALARY_MILLIONS,predict,real,estimate,times
0,Anthony Davis,C,23,0.505,11.8,28.0,NO,75,36.1,19.2,22.12,1.000000,2,1,3
1,Kyrie Irving,PG,24,0.473,3.2,25.2,CLE,72,35.1,13.5,17.64,1.000000,2,1,4
2,Danilo Gallinari,SF,28,0.447,5.2,18.2,DEN,63,33.9,11.4,15.05,1.000000,2,1,25
3,Evan Fournier,SG,24,0.439,3.1,17.2,ORL,68,32.9,8.5,17.00,1.000000,2,1,25
4,Eric Gordon,SG,28,0.406,2.7,16.2,HOU,75,31.0,8.2,12.39,1.000000,2,1,1
5,D'Angelo Russell,PG,20,0.405,3.5,15.6,LAL,63,28.7,9.9,5.33,0.000000,1,1,1
6,Nicolas Batum,SG,28,0.403,6.2,15.1,CHA,77,34.0,11.8,20.87,1.000000,2,1,1
7,Serge Ibaka,PF,27,0.480,6.8,14.8,ORL/TOR,79,30.7,10.6,12.25,1.000000,2,1,2
8,Jordan Clarkson,SG,24,0.445,3.0,14.7,LAL,82,29.2,8.4,12.50,1.000000,2,1,11
9,Khris Middleton,SF,25,0.450,4.2,14.7,MIL,29,30.7,10.4,15.20,1.000000,2,1,2


In [20]:
under_estimate_player = under_estimate_player.sort_values(by='times', ascending=False)
under_estimate_player

,PLAYER,POSITION,AGE,FG%,TRB,POINTS,TEAM,GP,MPG,PIE,SALARY_MILLIONS,predict,real,estimate,times
0,Isaiah Thomas,PG,27,0.463,2.7,28.9,BOS,76,33.8,16.1,6.59,2.000000,1,0,25
23,Trevor Booker,PF,29,0.516,8.0,10.0,BKN,71,24.7,12.7,9.25,2.000000,1,0,25
67,DeAndre Liggins,SG,28,0.387,1.7,2.5,CLE/DAL,62,12.5,4.6,1.58,1.000000,0,0,25
6,George Hill,PG,30,0.477,3.4,16.9,UTAH,49,31.5,12.8,8.00,2.000000,1,0,25
64,Andrew Bogut,C,32,0.469,8.1,2.9,CLE/DAL,27,21.6,9.2,0.24,1.840000,0,0,25
10,Dion Waiters,SG,25,0.424,3.3,15.8,MIA,46,30.1,10.0,2.90,2.000000,1,0,25
51,Lance Thomas,PF,28,0.398,3.1,6.0,NY,46,21.0,5.5,6.19,2.000000,1,0,25
46,Marcus Thornton,SG,29,0.400,2.3,6.6,WSH,33,17.4,6.9,0.98,2.000000,0,0,25
15,Bojan Bogdanovic,SF,27,0.445,3.4,13.7,BKN/WSH,81,25.7,9.1,3.73,2.000000,1,0,25
18,Trevor Ariza,SF,31,0.409,5.7,11.7,HOU,80,34.7,8.5,7.81,2.000000,1,0,25


In [21]:
over_estimate_player = over_estimate_player.sort_values(by='times', ascending=False)
over_estimate_player

,PLAYER,POSITION,AGE,FG%,TRB,POINTS,TEAM,GP,MPG,PIE,SALARY_MILLIONS,predict,real,estimate,times
48,John Henson,C,26,0.515,5.1,6.8,MIL,58,19.4,9.2,12.52,1.000000,2,1,25
2,Danilo Gallinari,SF,28,0.447,5.2,18.2,DEN,63,33.9,11.4,15.05,1.000000,2,1,25
3,Evan Fournier,SG,24,0.439,3.1,17.2,ORL,68,32.9,8.5,17.00,1.000000,2,1,25
53,Chandler Parsons,SF,28,0.338,2.5,6.2,MEM,34,19.9,5.6,22.12,0.760000,2,1,25
69,Miles Plumlee,C,28,0.478,2.1,2.5,MIL/CHA,45,10.8,4.0,12.50,1.000000,2,1,25
24,Allen Crabbe,SG,24,0.468,2.9,10.7,POR,79,28.5,7.4,18.50,1.000000,2,1,25
43,Manu Ginobili,SG,39,0.390,2.3,7.5,SA,69,18.7,8.7,14.00,1.000000,2,1,25
66,Dragan Bender,PF,19,0.354,2.4,3.4,PHX,43,13.3,3.3,4.28,0.000000,1,1,25
34,DeMarre Carroll,SF,30,0.400,3.8,8.9,TOR,72,26.1,7.1,14.20,1.000000,2,1,25
27,Tony Parker,PG,34,0.466,1.8,10.1,SA,63,25.2,9.3,14.45,1.000000,2,1,25


In [22]:
# all_false_player.to_csv('all_false_player.csv', sep=",")
# # all_false_player[under].to_csv('all_false_player[under].csv', sep=",")
# # all_false_player[over].to_csv('all_false_player[over].csv', sep=",")
# under_estimate_player.to_csv('under_estimate_player.csv', sep=",")
# over_estimate_player.to_csv('over_estimate_player.csv', sep=",")

In [23]:
#低估/高估球員各舉兩個球員作介紹
#over_estimate_player Ex:Chandler Parsons, Manu Ginobili
#under_estimate_player Ex: Giannis Antetokounmpo, Isaiah Thomas